In [ ]:
from rfdetr import RFDETRBase

model = RFDETRBase()

# Path to your checkpoint
checkpoint_path = "rfdetr_results/checkpoint_best_ema.pth"

model.train(
    dataset_dir="dataset",
    output_dir="rfdetr_results",
    
    # Continuation Logic
    resume=checkpoint_path,
    start_epoch=3,            
    epochs=15,                # Reduced to 15 to stay within your 4hr window
    
    # Corrected Settings
    resolution=448,           
    multi_scale=True,         
    expanded_scales=True,     # Fix: Change list to True
    
    # Memory Management
    batch_size=4,             
    grad_accum_steps=4,       
    amp=True,                 
    num_workers=2,            
    checkpoint_interval=1     
)

In [ ]:

from rfdetr import RFDETRBase

model = RFDETRBase()

model.train(
    dataset_dir="dataset",
    epochs=5,               # Set to 10; it will save progress even if it stops early
    output_dir="/content/rfdetr_results",
    batch_size=4,            # Processes 4 images at once (uses more VRAM, faster)
    grad_accum_steps=2,      # Only 2 steps to reach the target batch of 16
    checkpoint_interval=1    # Save every single epoch so no work is lost!
)


In [ ]:
from rfdetr import RFDETRBase

model = RFDETRBase()

# Use the Best EMA checkpoint for maximum accuracy
checkpoint_path = "rfdetr_results/checkpoint_best_ema.pth"

model.train(
    dataset_dir="dataset",
    output_dir="rfdetr_results",
    
    # Continuation Logic
    resume=checkpoint_path,
    start_epoch=3,            # Tells the logger to start counting from 3
    epochs=15,                
    
    # HIGH QUALITY SETTINGS (Restored)
    # Removing 'resolution' forces it back to the original high-quality default
    multi_scale=True,         
    expanded_scales=False,    # Keep this False to prevent hallucinations
    
    # MEMORY PROTECTION 
    # High resolution takes more VRAM. We use a smaller batch but more accumulation.
    batch_size=2,             # Only 2 images in memory at once
    grad_accum_steps=8,       # 2 x 8 = 16 (matches your successful original batch size)
    amp=True,                 
    num_workers=2,            
    checkpoint_interval=1     
)

In [ ]:
from rfdetr import RFDETRBase

model = RFDETRBase()

# Use the Best EMA checkpoint for maximum accuracy
checkpoint_path = "rfdetr_results/checkpoint_best_ema.pth"

model.train(
    dataset_dir="dataset",
    output_dir="rfdetr_results",
    
    # Continuation Logic
    resume=checkpoint_path,
    num_classes=14,  #or maybe 13
    start_epoch=3,            # Tells the logger to start counting from 3
    epochs=4,                
    # 2. DISABLE the things causing the jump in cardinality
    multi_scale=False,       # Force a single scale to stabilize the head
    expanded_scales=False,   # Turn off expanded scales (the 840px scale is too much right now)
    # MEMORY PROTECTION 
    # High resolution takes more VRAM. We use a smaller batch but more accumulation.
    batch_size=4,             # Only 2 images in memory at once
    grad_accum_steps=2,       # 2 x 4 = 8 (matches your successful original batch size)
    amp=True,                             
    checkpoint_interval=1     
)

In [ ]:
from rfdetr import RFDETRBase
import os

# 1. Initialize fresh model
model = RFDETRBase()

# 2. Point to your best weights
checkpoint_path = "rfdetr_results/checkpoint_best_ema.pth"

# 3. FORCE a fresh folder to prevent any hidden config issues
# We will use 'rfdetr_finetune' instead of 'rfdetr_results'
new_output_dir = "rfdetr_finetune"

model.train(
    dataset_dir="dataset",
    output_dir=new_output_dir, 
    
    # IMPORTANT: Use 'pretrain_weights' instead of 'resume'
    # This loads the patterns/learning but RESETs the learning rate and optimizer
    pretrain_weights=checkpoint_path,
    resume=None,               
    
    num_classes=14,
    start_epoch=0,           # Reset clock to 0 so the LR starts fresh
    epochs=2,               # Give it more time to stabilize
    
    # CRITICAL: Ultra-low LR to let the new head stabilize
    lr=0.000005,             # 5e-6 (even smaller than before)
    warmup_epochs=1.0,       # 1 full epoch of slow-start
    
    multi_scale=False,
    expanded_scales=False,
    batch_size=4,            # Smaller batch + more accumulation = more stability
    grad_accum_steps=2,      # Effective batch of 8
    amp=True,
    num_workers=2,
    checkpoint_interval=1
)